In [1]:
import numpy as np

In [2]:
class Day15():

    def __init__(self, input_link):
        self.link = input_link
        self.data_txt = open(input_link, 'r').read()

        board_flat, moves_flat = self.data_txt.split('\n\n')
        self.board = np.array([[letter for letter in elem] for elem in board_flat.split('\n')])
        self.moves = moves_flat.replace('\n', '')

        self.x, self.y = np.argwhere(self.board == '@')[0]
        return

    def display(self):
        print('\n'.join([''.join(list(line)) for line in self.board]))
        print()

    def move(self, direction):
        x, y = self.x, self.y
        elem = self.board[x, y]
        count = 0

        if direction == '>':
            while elem != '.':
                count += 1
                elem = self.board[x, y+count]
                if elem == '#':
                    return self
            self.board[self.x, self.y:self.y+count+1] = np.hstack((np.array(['.']), self.board[self.x, self.y:self.y+count]))
            self.y = self.y + 1

        if direction == '<':
            while elem != '.':
                count += 1
                elem = self.board[x, y-count]
                if elem == '#':
                    return self
            self.board[self.x, self.y-count:self.y+1] = np.hstack((self.board[self.x, self.y-count+1:self.y+1], np.array(['.'])))
            self.y = self.y - 1

        if direction == 'v':
            while elem != '.':
                count += 1
                elem = self.board[x+count, y]
                if elem == '#':
                    return self
            self.board[self.x:self.x+count+1, self.y] = np.hstack((np.array(['.']), self.board[self.x:self.x+count, self.y]))
            self.x = self.x + 1

        if direction == '^':
            while elem != '.':
                count += 1
                elem = self.board[x-count, y]
                if elem == '#':
                    return self
            self.board[self.x-count:self.x+1, self.y] = np.hstack((self.board[self.x-count+1:self.x+1, self.y], np.array(['.'])))
            self.x = self.x - 1

    def all_moves(self):
        for move in self.moves:
            self.move(move)

    def get_score(self):
        return np.sum([100*elem[0] + elem[1] for elem in np.argwhere(self.board == 'O')])


In [3]:
day15_test = Day15('other/day15_test.txt')
day15_test.all_moves()
day15_test.get_score()

10092

In [4]:
day15 = Day15('data/day15.txt')
day15.all_moves()
day15.get_score()

1499739

In [5]:
class Day15_2():

    def __init__(self, input_link):
        self.link = input_link
        self.data_txt = open(input_link, 'r').read()

        board_flat, moves_flat = self.data_txt.split('\n\n')
        board_flat_double = board_flat.replace('#', '##').replace('.', '..').replace('O', '[]').replace('@', '@.')
        self.board = np.array([[letter for letter in elem] for elem in board_flat_double.split('\n')])
        self.moves = moves_flat.replace('\n', '')

        self.x, self.y = np.argwhere(self.board == '@')[0]
        return

    def display(self):
        print('\n'.join([''.join(list(line)) for line in self.board]))
        print()

    def can_move(self, pos, direction, other = False):
        directions = {'>': (0, 1), '<': (0, -1), 'v': (1, 0), '^': (-1, 0)}
        somme = lambda x, y : tuple([x[i]+y[i] for i in range(len(x))])

        other_part = True
        if self.board[pos] == '[' and not other and direction in ['^', 'v']:
            other_part = self.can_move((pos[0], pos[1]+1), direction, True)
        if self.board[pos] == ']' and not other and direction in ['^', 'v']:
            other_part = self.can_move((pos[0], pos[1]-1), direction, True)

        check = other_part
        # Ce premier if est inutile mais balek
        if self.board[somme(pos, directions[direction])] == '.':
            check &= True
        elif self.board[somme(pos, directions[direction])] == '#':
            check &= False
        else :
            check &= self.can_move(somme(pos, directions[direction]), direction)

        return check

    def move(self, pos, direction, other = False):
        directions = {'>': (0, 1), '<': (0, -1), 'v': (1, 0), '^': (-1, 0)}
        somme = lambda x, y : tuple([x[i]+y[i] for i in range(len(x))])
        elem = self.board[pos]

        if elem == '@' and not self.can_move(pos, direction):
            return False

        if elem == '.':
            return True
        if elem == '#':
            return False
        if elem == '@':
            if self.move(somme(pos, directions[direction]), direction):
                self.board[pos], self.board[somme(pos, directions[direction])] = self.board[somme(pos, directions[direction])], self.board[pos]
                self.x, self.y = somme(pos, directions[direction])
                return True
            return False
        if elem == '[':
            if self.move(somme(pos, directions[direction]), direction) and (other or direction == '<' or self.move(somme(pos, directions['>']), direction, True)):
                self.board[pos], self.board[somme(pos, directions[direction])] = self.board[somme(pos, directions[direction])], self.board[pos]
                return True
            return False
        if elem == ']':
            if self.move(somme(pos, directions[direction]), direction) and (other or direction == '>' or self.move(somme(pos, directions['<']), direction, True)):
                self.board[pos], self.board[somme(pos, directions[direction])] = self.board[somme(pos, directions[direction])], self.board[pos]
                return True
            return False
        return True

    def all_moves(self):
        for move in self.moves:
            self.move((self.x, self.y), move)

    def get_score(self):
        return np.sum([100*elem[0] + elem[1] for elem in np.argwhere(self.board == '[')])


In [6]:
day15_2_test = Day15_2('other/day15_test.txt')
day15_2_test.all_moves()
day15_2_test.get_score()

9021

In [7]:
day15_2 = Day15_2('data/day15.txt')
day15_2.all_moves()
day15_2.get_score()

1522215